In [1]:
#5.1
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1621,  0.3344, -0.0975,  0.1109,  0.1315, -0.4677,  0.2131, -0.0483,
          0.2338, -0.2709],
        [-0.1159,  0.4168, -0.2490,  0.1038,  0.1473, -0.4194,  0.0854, -0.0716,
          0.1894, -0.2436]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0099,  0.1454,  0.2838, -0.3136,  0.0943, -0.1703, -0.0059,  0.1608,
          0.0474, -0.3629],
        [-0.0375,  0.1417,  0.1619, -0.2816,  0.0546, -0.0868,  0.0751,  0.0728,
          0.2317, -0.3503]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1283, -0.2505,  0.0692, -0.0353,  0.0963, -0.1223, -0.0376, -0.2937,
          0.1487,  0.2415],
        [-0.1198, -0.2287,  0.0902, -0.1079,  0.1324, -0.0883, -0.0382, -0.2606,
          0.1839,  0.2386]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.0785, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0739, grad_fn=<SumBackward0>)

In [10]:
#5.2

In [11]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2554],
        [-0.2658]], grad_fn=<AddmmBackward0>)

In [12]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1604, -0.0270,  0.0305,  0.0780,  0.0954, -0.0637, -0.1936, -0.3047]])), ('bias', tensor([-0.2656]))])


In [13]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2656], requires_grad=True)
tensor([-0.2656])


In [14]:
net[2].weight.grad == None

True

In [15]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
print(*[1])

1


In [18]:
net.state_dict()['2.bias'].data

tensor([-0.2656])

In [19]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4328],
        [-0.4328]], grad_fn=<AddmmBackward0>)

In [20]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [21]:
rgnet[0][1][0].bias.data

tensor([ 0.4470, -0.4885, -0.3025, -0.4820, -0.2540, -0.1423, -0.2662,  0.2187])

In [24]:
rgnet[0][1][0].state_dict()

OrderedDict([('weight',
              tensor([[-0.3459, -0.2524, -0.3705, -0.3608],
                      [ 0.0554, -0.4669,  0.3102, -0.2593],
                      [ 0.1479, -0.4595, -0.2734,  0.2630],
                      [ 0.4228, -0.2928, -0.4907,  0.3571],
                      [-0.0191, -0.4250, -0.1595,  0.4270],
                      [-0.0923, -0.4026,  0.1406, -0.2761],
                      [-0.1147, -0.1267, -0.3086,  0.4087],
                      [ 0.1983, -0.4089,  0.0151,  0.4727]])),
             ('bias',
              tensor([ 0.4470, -0.4885, -0.3025, -0.4820, -0.2540, -0.1423, -0.2662,  0.2187]))])

In [25]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0116, -0.0046,  0.0047,  0.0047]), tensor(0.))

In [29]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [30]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3235, -0.1640, -0.0403,  0.4865])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [31]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-8.1660,  0.0000,  0.0000, -5.3201],
        [-0.0000, -0.0000, -7.3922, -0.0000]], grad_fn=<SliceBackward0>)

In [39]:
a=3

In [44]:
a*= a>10

In [53]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [50]:
net[0].weight[:]

tensor([[-8.1660,  0.0000,  0.0000, -5.3201],
        [-0.0000, -0.0000, -7.3922, -0.0000],
        [-0.0000, -6.1722, -0.0000, -0.0000],
        [-0.0000,  0.0000, -9.8254, -6.3788],
        [ 5.8008,  0.0000, -9.6039, -0.0000],
        [-0.0000, -8.7447,  8.2795, -7.8203],
        [ 8.0795, -0.0000, -0.0000, -0.0000],
        [-0.0000,  6.6674, -0.0000, -6.1422]], grad_fn=<SliceBackward0>)

In [51]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[-8.1660,  0.0000,  0.0000, -5.3201],
                      [-0.0000, -0.0000, -7.3922, -0.0000],
                      [-0.0000, -6.1722, -0.0000, -0.0000],
                      [-0.0000,  0.0000, -9.8254, -6.3788],
                      [ 5.8008,  0.0000, -9.6039, -0.0000],
                      [-0.0000, -8.7447,  8.2795, -7.8203],
                      [ 8.0795, -0.0000, -0.0000, -0.0000],
                      [-0.0000,  6.6674, -0.0000, -6.1422]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[-0.0000, -0.0000, 5.5139, 0.0000, 0.0000, -0.0000, -0.0000, -0.0000]])),
             ('2.bias', tensor([0.]))])

In [54]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -4.3201])

In [55]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [56]:
#5.3